In [6]:
# standard imports
import numpy as np
import matplotlib.pyplot as plt 
import datetime
import sys
import geopandas as gpd
import pandas as pd
import seaborn as sns
import os
sns.set()

## Cryosat-2
You need to download the read-cryosat-2 tools and install them in your home directory.
In a terminal window, navigate to /home/jovyan/

Then:

`python3 -m pip install --user git+https://github.com/tsutterley/read-cryosat-2.git`

In [ ]:
from cryosat_toolkit.polygon import polygon

In [ ]:
ls

In [ ]:
cs2_bound = gpd.read_file('CryoSat_Mask_4.0.kmz')

In [ ]:
f1,a1 = plt.subplots(figsize=(10,5))
cs2_bound.plot(ax=a1)
a1.set_ylim(20,60)
a1.set_xlim(-130,-90)
f1.tight_layout()

In [ ]:
bounding_box_GM = [-108.19, 39.03, -108.18, 39.04]
lon_pts = [bounding_box_GM[0],bounding_box_GM[2]]
lat_pts = [bounding_box_GM[1],bounding_box_GM[3]]
gdf = gpd.GeoDataFrame()
gdf['lat'] = lat_pts
gdf['lon'] = lon_pts
dim = 1  # define the length of the side of the square
geoms = []
for index, row in gdf.iterrows():
    ln = row.lon
    lt = row.lat
    geom = Polygon([(ln, lt), ((ln + dim), lt), ((ln + dim), (lt - dim)), (ln, (lt - dim))])
    geoms.append(geom)

gdf['geometry'] = geoms 
gdf.crs = cs2_bound.crs
inter = gpd.overlay(cs2_bound,gdf,how='intersection')

In [ ]:
inter.plot()

In [ ]:
if not os.path.isfile('esa_cryosat_sync.py'):
    !wget https://raw.githubusercontent.com/tsutterley/read-cryosat-2/main/scripts/esa_cryosat_sync.py
else:
    print('ESA_cryosat_sync.py already present')

In [4]:
pwd

'/home/jovyan/integration_station/contributors/max'

In [11]:
import esa_cryosat_sync

SyntaxError: invalid syntax (<ipython-input-11-bc8eb650cde7>, line 1)

In [12]:
sys.path

['/home/jovyan/integration_station/contributors/max',
 '/srv/conda/envs/notebook/lib/python38.zip',
 '/srv/conda/envs/notebook/lib/python3.8',
 '/srv/conda/envs/notebook/lib/python3.8/lib-dynload',
 '',
 '/home/jovyan/.local/lib/python3.8/site-packages',
 '/srv/conda/envs/notebook/lib/python3.8/site-packages',
 '/srv/conda/envs/notebook/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/srv/conda/envs/notebook/lib/python3.8/site-packages/IPython/extensions',
 '/home/jovyan/.ipython']